"Bitcoin is the currency of the Internet: a distributed, worldwide, decentralized digital money. Unlike traditional currencies such as dollars, bitcoins are issued and managed without any central authority whatsoever: there is no government, company, or bank in charge of Bitcoin. As such, it is more resistant to wild inflation and corrupt banks. With Bitcoin, you can be your own bank." -- [r/bitcoin](https://www.reddit.com/r/Bitcoin/)

This brief summary of bitcoin is as good as it gets. It has been around for 10 years now. It has catched anyones attention at least once in this period and has been centerpoint of media, internet and discussions all over the world. What most common people are interested in is the volatile worth of bitcoin and the big profits it has been possible to make over the years with just the right timing of investment. 

Bitcoin is mainly obtained and trained in big exchange sites such as coinbase and bitstamp. The main currency it is based on is USD and can be exchanged real-time. 



In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


In [3]:
df = pd.read_csv('bitstamp2012-2019.csv')

In [4]:
df.head(10)

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1325318220,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1325318280,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1325318340,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1325318400,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1325318460,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
pd.to_datetime(df['Timestamp'],unit='s').dt.hour

0           7
1           7
2           7
3           7
4           7
5           7
6           7
7           7
8           8
9           8
10          8
11          8
12          8
13          8
14          8
15          8
16          8
17          8
18          8
19          8
20          8
21          8
22          8
23          8
24          8
25          8
26          8
27          8
28          8
29          8
           ..
3997667    23
3997668    23
3997669    23
3997670    23
3997671    23
3997672    23
3997673    23
3997674    23
3997675    23
3997676    23
3997677    23
3997678    23
3997679    23
3997680    23
3997681    23
3997682    23
3997683    23
3997684    23
3997685    23
3997686    23
3997687    23
3997688    23
3997689    23
3997690    23
3997691    23
3997692    23
3997693    23
3997694    23
3997695    23
3997696     0
Name: Timestamp, Length: 3997697, dtype: int64

In [26]:
df['datetime'] = pd.to_datetime(df['Timestamp'],unit='s')
df['date'] = pd.to_datetime(df['Timestamp'],unit='s').dt.date
df['year'] = pd.to_datetime(df['Timestamp'],unit='s').dt.year
df['month'] = pd.to_datetime(df['Timestamp'],unit='s').dt.month
df['day'] = pd.to_datetime(df['Timestamp'],unit='s').dt.day
df['hour'] = pd.to_datetime(df['Timestamp'],unit='s').dt.hour
df['weekday'] = pd.to_datetime(df['Timestamp'],unit='s').dt.weekday_name


In [27]:
df.head(5)

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,datetime,date,year,month,day,weekday,hour
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31 07:52:00,2011-12-31,2011,12,31,Saturday,7
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:53:00,2011-12-31,2011,12,31,Saturday,7
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:54:00,2011-12-31,2011,12,31,Saturday,7
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:55:00,2011-12-31,2011,12,31,Saturday,7
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:56:00,2011-12-31,2011,12,31,Saturday,7
